In [1]:
# Full documentation at http://physical-validation.readthedocs.io/
import physical_validation as pv

In [2]:
# Create a LAMMPS parser
parser = pv.data.LammpsParser()

###########################################################################
# WARNING: The LAMMPS parser is an experimental feature under current     #
#          development. You can help us to improve it by reporting errors #
#          at https://github.com/shirtsgroup/physical_validation          #
#          Thank you!                                                     #
###########################################################################


In [3]:
# In this example, we will only look at the simulations performed under
# NVT conditions using `fix nvt` (Nose-Hoover thermostat). There are two simulations:
# One ran at 300K, and one ran at 308K.
ensemble_1 = pv.data.EnsembleData(
    ensemble='NVT',
    natoms=100*3,
    volume=20**3,
    temperature=300
)
ensemble_2 = pv.data.EnsembleData(
    ensemble='NVT',
    natoms=100*3,
    volume=20**3,
    temperature=308
)
dir_1 = 'nh_1'
dir_2 = 'nh_2'

In [4]:
# We can now use the LAMMPS parser to create simulation results for the two NVT
# simulations, using the ensemble definitions created before:
result_1 = parser.get_simulation_data(
    ensemble=ensemble_1,
    in_file=dir_1 + '/water.in',
    log_file=dir_1 + '/log.lammps',
    data_file=dir_1 + '/water.lmp',
    dump_file=dir_1 + '/dump.atom'
)
result_2 = parser.get_simulation_data(
    ensemble=ensemble_2,
    in_file=dir_2 + '/water.in',
    log_file=dir_2 + '/log.lammps',
    data_file=dir_2 + '/water.lmp',
    dump_file=dir_2 + '/dump.atom'
)


/home/pascal/physval/pyenv/3.6-jupyter/lib/python3.6/site-packages/physical_validation/data/lammps_parser.py:457: UserWarning: Multiple runs found in log file. Assumed prior runs were equilibration and used only last run.
  warnings.warn('Multiple runs found in log file. Assumed prior runs '
/home/pascal/physval/pyenv/3.6-jupyter/lib/python3.6/site-packages/physical_validation/data/lammps_parser.py:168: UserWarning: Found `fix shake` or `fix rattle`. Reading of constraints is not implemented yet. Please set self.system.nconstraints manually!
  warnings.warn('Found `fix shake` or `fix rattle`. Reading of '
/home/pascal/physval/pyenv/3.6-jupyter/lib/python3.6/site-packages/physical_validation/data/observable_data.py:208: UserWarning: Mismatch in number of frames. Setting `nframes = None`.
  warnings.warn('Mismatch in number of frames. '


In [5]:
# The LAMMPS parser is currently not able to set the number of constraints due
# to `fix shake` or `fix rattle` commands (see output of previous commands).
# We'll hence add this here (3 constraints per molecule):
result_1.system.nconstraints=300
result_2.system.nconstraints=300

In [6]:
# As with the other parsers, we can now test, for example, the kinetic energy using the
# created simulation result data structure, where
# the first input is the simulation results read in,
# `strict` determines whether we test the full distribution (True)
# or only determine the mean and the variance of the distribution (False),
# `verbosity` sets the level of detail of the output  (with verbosity=0
# being quiet and verbosity=3 being the most chatty), and the filename is being used to
# plot the resulting distribution for visual inspection.
print('==> Kinetic energy test of simulation ' + dir_1)
pv.kinetic_energy.distribution(result_1, strict=False, verbosity=2,
                               filename='ke_lammps_vr_NVT_1')
print('==> Kinetic energy test of simulation ' + dir_2)
pv.kinetic_energy.distribution(result_2, strict=False, verbosity=2,
                               filename='ke_lammps_vr_NVT_2')

==> Kinetic energy test of simulation nh_1
After equilibration, decorrelation and tail pruning, 98.01% (197 frames) of original Kinetic energy remain.
Kinetic energy distribution check (non-strict)
Analytical distribution (T=300.00 K):
 * mu: 178.85 kcal/mol
 * sigma: 10.33 kcal/mol
Trajectory:
 * mu: 177.74 +- 0.68 kcal/mol
   T(mu) = 298.13 +- 1.14 K
 * sigma: 10.37 +- 0.66 kcal/mol
   T(sigma) = 301.16 +- 19.17 K
==> Kinetic energy test of simulation nh_2
After equilibration, decorrelation and tail pruning, 100.00% (201 frames) of original Kinetic energy remain.
Kinetic energy distribution check (non-strict)
Analytical distribution (T=308.00 K):
 * mu: 183.62 kcal/mol
 * sigma: 10.60 kcal/mol
Trajectory:
 * mu: 182.78 +- 0.75 kcal/mol
   T(mu) = 306.59 +- 1.26 K
 * sigma: 10.26 +- 0.46 kcal/mol
   T(sigma) = 298.22 +- 13.26 K


(1.11858538588982, 0.7370659213215571)

In [7]:
# We can also test the distribution of the potential energy. While the first two
# inputs to the tests are the parsed simulation results, `verbosity` sets the level of
# detail of the output  (with verbosity=0 being quiet and verbosity=3 being the most chatty),
# and the filename is being used to plot the resulting distribution for visual inspection.
print('==> Potential energy test')
pv.ensemble.check(result_1, result_2,
                  verbosity=2, filename='pe_lammps_vr_NVT')

==> Potential energy test
Analytical slope of ln(P_2(U)/P_1(U)): 0.04356879
After equilibration, decorrelation and tail pruning, 25.37% (51 frames) of original Trajectory 1 remain.
After equilibration, decorrelation and tail pruning, 27.36% (55 frames) of original Trajectory 2 remain.
Overlap is 78.4% of trajectory 1 and 92.7% of trajectory 2.
A rule of thumb states that a good overlap is found when dT/T = (2*kB*T)/(sig),
where sig is the standard deviation of the energy distribution.
For the current trajectories, dT = 8.0, sig1 = 16.3 and sig2 = 15.0.
According to the rule of thumb, given T1, a good dT is dT = 21.9, and
                                given T2, a good dT is dT = 25.1.
Rule of thumb estimates that dT = 23.5 would be optimal (currently, dT = 8.0)
Linear Fit Analysis (analytical error)
Free energy
    280.16085 +/- 259.03565
Estimated slope                  |  True slope
    0.284965  +/- 0.263054       |  0.043569 
    (0.92 quantiles from true slope)
Estimated dT      

[0.6261178879899326]